In [1]:
# placeholder jupyter notebook for the final project

In [2]:
import os

import pandas as pd
import numpy as np

# Assign spreadsheet filename to 'file'
Sales_file = 'Louisville Sales.xlsx'

# Load spreadsheet
salesxl = pd.ExcelFile(Sales_file)

# Print the sheet names
print(salesxl.sheet_names)

['sheet1']


In [3]:
# Load a sheet into a DataFrame by name: df1
df1 = salesxl.parse('sheet1')

Rent_file = 'Rental data.xlsx'
rentxl = pd.ExcelFile(Rent_file)
print(rentxl.sheet_names)

['sheet1']


In [4]:
df2 = rentxl.parse('sheet1')

# writer = pd.ExcelWriter('practice.xlsx', engine='xlsxwriter')
# df2.to_excel(writer, 'Sheet1')
# writer.save()

In [5]:
df1.head()

,List Number,Book Section,Property Sub Type,Listing Date,Sold Date,Contract Acceptance Date,Original List Price,Listing Price,Sold Price,Area,...,End Date,HOA Y/N,Stories,# Upper Floor Bedrooms,First Floor Laundry,Monthly Maintenance,Seller Concessions,HOA Annual $,Closing Costs and Prepaids,Days on Market
0,1573919,Single Family Residence,SF,2020-11-12,2020-12-30,2020-12-18,189950.0,189950.0,189900.0,5,...,2021-05-12,Y,1,0,N,NaN,Y,400.0,4600.0,36
1,1572930,Single Family Residence,SF,2020-10-30,2020-12-31,2020-11-13,184900.0,182900.0,175000.0,5,...,2021-10-30,N,1,0,Y,NaN,N,0.0,NaN,14
2,1574676,Single Family Residence,SF,2020-11-02,2020-11-20,2020-11-02,290000.0,290000.0,290000.0,9,...,2020-11-30,N,1,0,N,NaN,N,0.0,NaN,0
3,1574083,Condominium,CP,2020-11-16,2021-01-08,2020-12-21,69900.0,67000.0,62000.0,3,...,2021-02-15,Y,1,0,N,NaN,N,3153.0,NaN,35
4,1572788,Condominium,CP,2020-10-29,2020-11-30,2020-10-30,115000.0,115000.0,115000.0,7,...,2021-04-29,Y,1,0,Y,NaN,N,2328.0,NaN,1


In [6]:
# calculating subdivision level average price per sqft and average age and average size for both rental and sales

groupped_sales_df = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})

sales_count = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
groupped_sales_df = pd.merge(groupped_sales_df, sales_count, on="Subdivision/Condo")
groupped_sales_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count
0,1400 Willow,335.096667,40.000000,2584.666667,3
1,2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000,1
2,550 LOFTS,189.930000,97.000000,1211.000000,1
3,ABBEY WOODS,131.650000,24.500000,1871.500000,2
4,ABERDEEN,208.615714,79.214286,2469.500000,14
5,ACADEMY ESTATES,189.814000,6.400000,3964.800000,5
6,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3
7,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4
8,ADAMS HILL,102.107500,41.750000,1174.000000,4
9,ADAMS RUN,117.815000,18.045455,2193.500000,22


In [7]:
groupped_rental_df = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})


rent_count = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='rent_count')

groupped_rental_df = pd.merge(groupped_rental_df, rent_count, on="Subdivision/Condo")

groupped_rental_df

,Subdivision/Condo,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,2000 SPRING CONDOMIN,1.030000,102.000000,1600.000000,1
1,ABERDEEN,1.155000,74.000000,1873.000000,2
2,ACADEMY RIDGE,0.894000,15.600000,2535.200000,5
3,ACADEMY RIDGE AT LANDIS LAKE,1.000000,17.000000,2030.000000,1
4,ACADEMY RIDGE ESTATES,0.830000,18.000000,1900.000000,2
5,ADAMS RUN,0.890000,18.000000,1998.666667,3
6,AIRVIEW,0.907143,60.428571,835.000000,7
7,ALGONQUIN PLACE,0.680000,65.000000,1025.000000,1
8,ALHAMBRA HEIGHTS,1.300000,70.500000,1653.500000,2
9,ALIA,0.880000,21.000000,5030.000000,1


In [8]:
# Merge the 2 dataframes and only keep the subdivisions where there is both sales and rental data available 

subdivision_df = pd.merge(groupped_sales_df, groupped_rental_df, on="Subdivision/Condo", how='inner')
subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count
0,2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000,1,1.030000,102.000000,1600.000000,1
1,ABERDEEN,208.615714,79.214286,2469.500000,14,1.155000,74.000000,1873.000000,2
2,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3,0.894000,15.600000,2535.200000,5
3,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4,1.000000,17.000000,2030.000000,1
4,ADAMS RUN,117.815000,18.045455,2193.500000,22,0.890000,18.000000,1998.666667,3
5,AIRVIEW,154.500000,70.000000,1255.000000,1,0.907143,60.428571,835.000000,7
6,ALGONQUIN PLACE,65.457273,68.090909,1252.272727,11,0.680000,65.000000,1025.000000,1
7,ALHAMBRA HEIGHTS,201.675714,74.285714,2480.142857,7,1.300000,70.500000,1653.500000,2
8,ALLSTON,164.222500,77.000000,1236.750000,4,1.000000,69.500000,1150.000000,2
9,ALPHA GARDENS,103.210000,49.000000,1182.000000,1,0.760000,52.000000,2495.000000,1


In [9]:
subdivision_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697 entries, 0 to 696
Data columns (total 9 columns):
Subdivision/Condo     697 non-null object
Average_Sold_Price    697 non-null float64
Average_Sold_Age      697 non-null float64
Average_Sold_SqFt     697 non-null float64
sales_count           697 non-null int64
Average_Rent_Price    693 non-null float64
Average_Rent_Age      697 non-null float64
Average_Rent_SqFt     697 non-null float64
rent_count            697 non-null int64
dtypes: float64(6), int64(2), object(1)
memory usage: 54.5+ KB


In [14]:
subdivision_df['rent_sales_ratio'] = pd.eval('subdivision_df.Average_Rent_Price/subdivision_df.Average_Sold_Price')*1000
subdivision_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697 entries, 0 to 696
Data columns (total 10 columns):
Subdivision/Condo     697 non-null object
Average_Sold_Price    697 non-null float64
Average_Sold_Age      697 non-null float64
Average_Sold_SqFt     697 non-null float64
sales_count           697 non-null int64
Average_Rent_Price    693 non-null float64
Average_Rent_Age      697 non-null float64
Average_Rent_SqFt     697 non-null float64
rent_count            697 non-null int64
rent_sales_ratio      693 non-null float64
dtypes: float64(7), int64(2), object(1)
memory usage: 59.9+ KB


In [30]:
# exclude items from the dataframe where the average SqFt and/or Age of the houses sold are very different (more than 30% higher
# or lower) from the average age and/of SqFt of the rented homes -> the Rental and Sales price comparison is not relevant

subdivision_df['rent_sales_age_percent'] = pd.eval('abs((subdivision_df.Average_Sold_Age-subdivision_df.Average_Rent_Age)/subdivision_df.Average_Sold_Age)')
subdivision_df['rent_sales_SqFt_percent'] = pd.eval('abs((subdivision_df.Average_Sold_SqFt-subdivision_df.Average_Rent_SqFt)/subdivision_df.Average_Sold_SqFt)')
subdivision1_df = subdivision_df.drop(subdivision_df[subdivision_df['rent_sales_age_percent'] > .3].index)
reduced_subdivision_df = subdivision1_df.drop(subdivision1_df[subdivision1_df['rent_sales_SqFt_percent'] > .3].index)
reduced_subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
1,ABERDEEN,208.615714,79.214286,2469.500000,14,1.155000,74.000000,1873.000000,2,5.536496,0.065825,0.241547
2,ACADEMY RIDGE,144.336667,16.666667,2128.333333,3,0.894000,15.600000,2535.200000,5,6.193852,0.064000,0.191167
3,ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,4,1.000000,17.000000,2030.000000,1,6.887645,0.014493,0.237272
4,ADAMS RUN,117.815000,18.045455,2193.500000,22,0.890000,18.000000,1998.666667,3,7.554216,0.002519,0.088823
6,ALGONQUIN PLACE,65.457273,68.090909,1252.272727,11,0.680000,65.000000,1025.000000,1,10.388456,0.045394,0.181488
8,ALLSTON,164.222500,77.000000,1236.750000,4,1.000000,69.500000,1150.000000,2,6.089300,0.097403,0.070144
10,ALTA VISTA,153.835000,46.000000,5449.000000,2,1.210000,50.000000,4789.000000,1,7.865570,0.086957,0.121123
12,ANCHORAGE,206.417586,63.793103,4508.310345,29,1.095000,47.833333,3816.166667,6,5.304781,0.250180,0.153526
13,ANCHORAGE PIKE,156.790000,33.000000,1722.000000,1,1.240000,34.000000,1676.000000,3,7.908668,0.030303,0.026713
16,ARLINGTON,202.191429,78.714286,1874.857143,7,1.364000,77.000000,2036.000000,5,6.746082,0.021779,0.085949


In [31]:
# show the subdivisions with the highest rent(per sqft)/sales(per sqft) ratio -> where you get the most bang for your buck
# if investing in a rental property

sorted_subdivision_df = reduced_subdivision_df.sort_values(['rent_sales_ratio'], ascending=False)
sorted_subdivision_df

,Subdivision/Condo,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,sales_count,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt,rent_count,rent_sales_ratio,rent_sales_age_percent,rent_sales_SqFt_percent
371,LASALLE PLACE,34.960000,111.000000,758.000000,1,0.880000,118.000000,906.000000,1,25.171625,0.063063,0.195251
341,INNWOOD,51.577500,39.000000,538.000000,4,1.250000,38.000000,538.000000,1,24.235374,0.025641,0.000000
637,WESTMEADE,29.990000,91.000000,1142.000000,1,0.720000,94.000000,1040.000000,1,24.008003,0.032967,0.089317
192,DULWORTH JC,72.133333,66.666667,1187.333333,3,1.120000,61.000000,1114.000000,1,15.526802,0.085000,0.061763
178,DEARBORN,52.955000,71.500000,1205.000000,4,0.816667,86.666667,1211.000000,3,15.421899,0.212121,0.004979
78,BROAD ACRES,76.000000,69.500000,700.000000,2,1.070000,68.000000,700.000000,1,14.078947,0.021583,0.000000
376,LENOX,73.103333,76.666667,940.933333,15,1.000000,69.400000,730.400000,5,13.679267,0.094783,0.223749
37,BAYBERRY CONDOS,62.300000,35.000000,915.000000,1,0.840000,35.000000,1011.000000,1,13.483146,0.000000,0.104918
462,PARKDALE,73.091250,72.750000,1186.500000,8,0.917500,78.200000,929.400000,5,12.552802,0.074914,0.216688
255,GLASSWORKS,99.753500,110.000000,1303.750000,20,1.220000,110.000000,1000.000000,1,12.230147,0.000000,0.232982
